In [1]:
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np

# Load images from a pickle file
with open("stl_test.pickle", "rb") as tr_images_file:
    test = pickle.load(tr_images_file)
with open("stl_train.pickle", "rb") as tr_images_file:
    train = pickle.load(tr_images_file)
with open("stl_val.pickle", "rb") as tr_images_file:
    new = pickle.load(tr_images_file)
with open("stl_deg.pickle", "rb") as tr_images_file:
    deg = pickle.load(tr_images_file)




In [2]:
def unpk(data):

    x=[]
    y=[]
    c=0
    tot=[]
    for pair in data:
        try:
            img=np.array(pair[0]).reshape((96,96,3))
            x.append(img)
            y.append(pair[1])
        except Exception as e:
            c=c+1
    
    print("exceptions")
    print(c)
    return x, y

#unpk opens the pickle files and separates images from labels 

x_train,y_trai=unpk(train)
x_test,y_tes=unpk(test)
x_new, y_ne=unpk(new)
x_deg, y_deg=unpk(deg)

exceptions
0
exceptions
0
exceptions
0
exceptions
0


In [3]:
print("train data:")
print(len(x_train))
print("test data:")
print(len(x_test))
print("new unseen data:")
print(len(x_new))
print("extra training label, class truck:")
print(len(x_deg))

train data:
5850
test data:
2925
new unseen data:
2925
extra training label, class truck:
1300


In [4]:
#since in precedence the drift was simulated by adding an extra class we reintegrate the class with
#other data
for i in range(len(x_deg)):
    if i<650:
        x_train.append(x_deg[i])
        y_trai.append(y_deg[i])
    elif i>=650 and i<975:
        x_test.append(x_deg[i])
        y_tes.append(y_deg[i])
    else:
        x_new.append(x_deg[i])
        y_ne.append(y_deg[i])
        
print(len(x_train))
print(len(x_test))
print(len(x_new))
#print(len(x_deg))

6500
3250
3250


In [5]:
from PIL import Image, ImageFilter, ImageDraw, ImageOps
import numpy as np
from IPython.display import display

def blur_inside_percentage_of_image(image_array, percentage):
    # Open the image
    #img = Image.open(image_path)
    img=Image.fromarray(image_array)
    # Get the dimensions of the image
    width, height = img.size
    area=width*height

    # Calculate the spot size based on the percentage of the image dimensions
    circ_area=int(area* percentage / 100)
    spot_size_rad=int(np.sqrt(circ_area/3.14))
    spot_size=2*spot_size_rad
    # Define the coordinates for the center of the image
    center_x = width // 2
    center_y = height // 2

    # Create a circular mask
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((center_x - spot_size // 2, center_y - spot_size // 2,
                  center_x + spot_size // 2, center_y + spot_size // 2), fill=255)

    # Invert the circular mask
    inverted_mask = ImageOps.invert(mask)

    # Blur the inside of the circular area using the inverted mask
    blurred_img = img.filter(ImageFilter.GaussianBlur(radius=8))  # Adjust radius as needed
    blurred_img.paste(img, mask=inverted_mask)
    final_array = np.array(blurred_img).reshape((96,96,3))
    
    return final_array
#in this step we blur the images
#if prc=0 then the images have no blur and belong to the new unseen data set
prc=20  # Adjust the percentage as needed, this parameter impacts the area blurred

blur_images=[]
for i in range(len(x_new)):
    if i%500==0:
        print(i)
    tmp=x_new[i]
    blur_images.append(blur_inside_percentage_of_image(x_new[i], prc))


0
500
1000
1500
2000
2500
3000


In [6]:

x_train=np.array(x_train)
x_test=np.array(x_test)
x_blur=np.array(blur_images)
#x_deg=np.array(b_t_images)
y_train=np.array(y_trai)
y_test=np.array(y_tes)
y_blur=np.array(y_ne)

In [7]:
def create_ds(im, lab):
    ds=[]
    for i in range(len(im)):
        ds.append((im[i],lab[i]))

    return ds



train_ds=create_ds(x_train, y_train)
test_ds=create_ds(x_test, y_test)
blur_ds=create_ds(x_blur, y_blur)


print("train")   
print(len(train_ds))
print("test")
print(len(test_ds))
print("blur")
print(len(blur_ds))

train
6500
test
3250
blur
3250


In [8]:
from transformers import ViTModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
import torch.nn.functional as F

class ViTForImageClassification(nn.Module):
    def __init__(self, num_labels=10):
        super(ViTForImageClassification, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, pixel_values, labels):
        outputs = self.vit(pixel_values=pixel_values)
        output = self.dropout(outputs.last_hidden_state[:,0])
        logits = self.classifier(output)

        loss = None
        if labels is not None:
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        if loss is not None:
          return logits, loss.item()
        else:
          return logits, None
    
    def emb_extr_new_v1(self, pixel_values):
        
        outputs = self.vit(pixel_values=pixel_values)
        #output = self.dropout(outputs.last_hidden_state[:,0])
        #for layer in model.children():
        
        return outputs.last_hidden_state[:,0]

In [9]:
EPOCHS = 5
BATCH_SIZE = 50
LEARNING_RATE = 2e-5


In [10]:
from transformers import ViTFeatureExtractor
import torch.nn as nn
import torch
# Define Model
model = ViTForImageClassification(10)    
# Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# Cross Entropy Loss
loss_func = nn.CrossEntropyLoss()
# Use GPU if available  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.cuda() 

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [12]:
import torch.utils.data as data
from torch.autograd import Variable
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score


#load data into train and test loader and start traininig phase
train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_loader  = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 
print("training...")
# Train the model
for epoch in range(EPOCHS):        
    for step, (x, y) in enumerate(train_loader):
        # Change input array into list with each batch being one element
        x = np.split(np.squeeze(np.array(x)), BATCH_SIZE)
        # Remove unecessary dimension
        for index, array in enumerate(x):
            x[index] = np.squeeze(array)
        # Apply feature extractor, stack back into 1 tensor and then convert to tensor
        x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
        # Send to GPU if available
        x, y  = x.to(device), y.to(device)
        b_x = Variable(x)   # batch x (image)
        b_y = Variable(y)   # batch y (target)
        # Feed through model
        output, loss = model(b_x, None)
        # Calculate loss
        if loss is None: 
            loss = loss_func(output, b_y)   
            optimizer.zero_grad()           
            loss.backward()                 
            optimizer.step()

        if step % 50 == 0:
            # Get the next batch for testing purposes
            test = next(iter(test_loader))
            test_x = test[0]
            # Reshape and get feature matrices as needed
            test_x = np.split(np.squeeze(np.array(test_x)), BATCH_SIZE)
            for index, array in enumerate(test_x):
                test_x[index] = np.squeeze(array)
            test_x = torch.tensor(np.stack(feature_extractor(test_x)['pixel_values'], axis=0))
            # Send to appropirate computing device
            test_x = test_x.to(device)
            test_y = test[1].to(device)
            # Get output (+ respective class) and compare to target
            test_output, loss = model(test_x, test_y)
            test_output = test_output.argmax(1)
            # Calculate Accuracy
            accuracy = (test_output == test_y).sum().item() / BATCH_SIZE
            print('Epoch: ', epoch, '| train loss: %.4f' % loss, '| test accuracy: %.2f' % accuracy)


training...
Epoch:  0 | train loss: 2.3218 | test accuracy: 0.08
Epoch:  0 | train loss: 1.7114 | test accuracy: 0.96
Epoch:  0 | train loss: 0.9529 | test accuracy: 1.00
Epoch:  1 | train loss: 0.7638 | test accuracy: 0.98
Epoch:  1 | train loss: 0.5512 | test accuracy: 0.98
Epoch:  1 | train loss: 0.3581 | test accuracy: 1.00
Epoch:  2 | train loss: 0.3394 | test accuracy: 0.98
Epoch:  2 | train loss: 0.2670 | test accuracy: 1.00
Epoch:  2 | train loss: 0.2493 | test accuracy: 1.00
Epoch:  3 | train loss: 0.2241 | test accuracy: 0.98
Epoch:  3 | train loss: 0.1976 | test accuracy: 0.96
Epoch:  3 | train loss: 0.3393 | test accuracy: 0.94
Epoch:  4 | train loss: 0.2385 | test accuracy: 0.98
Epoch:  4 | train loss: 0.2005 | test accuracy: 0.98
Epoch:  4 | train loss: 0.1604 | test accuracy: 0.98


In [13]:
from torchvision import transforms


model.eval()

# Load the test dataset from pickle

#test_data =test_ds

# Define a custom dataset class for the test set
class CustomTestDataset():
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, label = self.data[index]
        # Perform any necessary preprocessing on the image
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = transform(Image.fromarray(image))
        return image, label

# Create a DataLoader for the test dataset
test_dataset = CustomTestDataset(test_ds)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Perform inference on the test set
all_labels = []
all_predictions = []

print("evaluation...")
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs, loss = model(inputs, labels)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        #all_embeddings.append(embeddings.cpu().numpy())

# Evaluate the model

print(confusion_matrix(all_labels, all_predictions))

# Print classification report
print("Classification Report:")
print(classification_report(all_labels, all_predictions))



evaluation...
[[310   0   9   0   0   1   2   0   0   3]
 [  0 306   0  17   0   2   0   0   0   0]
 [ 17   0 288   0   1   1   5   0   7   6]
 [  0   4   0 317   0   1   0   3   0   0]
 [  5   0   1   0 309   3   7   0   0   0]
 [  0   1   0   1   0 319   0   4   0   0]
 [  5   0   0   0   3   1 315   0   0   1]
 [  0   0   0   3   0   3   0 319   0   0]
 [  1   0   1   1   0   0   2   0 314   6]
 [  4   0   1   0   0   0   1   0   1 318]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       325
           1       0.98      0.94      0.96       325
           2       0.96      0.89      0.92       325
           3       0.94      0.98      0.95       325
           4       0.99      0.95      0.97       325
           5       0.96      0.98      0.97       325
           6       0.95      0.97      0.96       325
           7       0.98      0.98      0.98       325
           8       0.98      0.97      0.97  

In [14]:
blur_dataset = CustomTestDataset(blur_ds)
blur_dataloader = data.DataLoader(blur_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Perform inference on the test set
all_labels = []
all_predictions = []

print("start evaluation...")
with torch.no_grad():
    for inputs, labels in blur_dataloader:
        outputs, loss = model(inputs, labels)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        #all_embeddings.append(embeddings.cpu().numpy())

# Evaluate the model

print(confusion_matrix(all_labels, all_predictions))

# Print classification report
print("Classification Report:")
print(classification_report(all_labels, all_predictions))


inizio evaluation...
[[274   0   6   3  10   5  11   1   3  12]
 [  0 276   0  34   0  15   0   0   0   0]
 [ 33   0 224   1  18   4   8   0  21  16]
 [  0   1   0 291   0  11   0  22   0   0]
 [  0   0   2   2 307  13   1   0   0   0]
 [  0   0   0   2   0 302   0  20   0   1]
 [  6   0   6   0  29   3 273   1   3   4]
 [  0   0   0   2   0   9   0 314   0   0]
 [  0   0   1   2   3   3   2   1 301  12]
 [  5   0   2   0  10   2   1   0   1 304]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       325
           1       1.00      0.85      0.92       325
           2       0.93      0.69      0.79       325
           3       0.86      0.90      0.88       325
           4       0.81      0.94      0.87       325
           5       0.82      0.93      0.87       325
           6       0.92      0.84      0.88       325
           7       0.87      0.97      0.92       325
           8       0.91      0.93     

In [38]:
def embedding_extraction(modello, dataloader):
    
    embeddings=[]
    print("start embedding extraction...")

    with torch.no_grad():
        embeddings = []

        for images, _ in dataloader:  # Adjust this loop based on your data loading
            outputs = modello.emb_extr_new_v1(images)
            #emb_extr_new_v1
            embeddings.append(outputs)
            
    embeddings=np.array(embeddings, dtype='object')
            
    return embeddings

def into_dataloader(ds):
    dataset = CustomTestDataset(ds)
    dataloader = data.DataLoader(dataset, batch_size=50, shuffle=False)
    
    return dataloader




model.eval()



print("train")
train_emb=embedding_extraction(model, into_dataloader(train_ds))
print("test")
test_emb=embedding_extraction(model, into_dataloader(test_ds))
print("blur")
blur_emb=embedding_extraction(model, into_dataloader(blur_ds))

print("done")

blur
inizio extraction embedding...


/tmp/ipykernel_150/3811817293.py:14: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  embeddings=np.array(embeddings, dtype='object')


train
inizio extraction embedding...
test
inizio extraction embedding...
done


In [39]:
def prediction_extraction(modello, dataloader):
    all_labels = []
    all_predictions = []

    print("start evaluation...")
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs, loss = modello(inputs, labels=None)
            _, predictions = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            #all_embeddings.append(embeddings.cpu().numpy())
            
    return all_predictions, all_labels




print("predictions...")
print("train")
train_pred, train_lab=prediction_extraction(model, into_dataloader(train_ds))
print("test")
test_pred, test_lab=prediction_extraction(model, into_dataloader(test_ds))
print("blur")
blur_pred, blur_lab=prediction_extraction(model, into_dataloader(blur_ds))
#print("deg")
#deg_pred, deg_lab=prediction_extraction(model, into_dataloader(drift))
print("done")

predictions...
train
inizio evaluation...
test
inizio evaluation...
blur
inizio evaluation...
done


In [40]:
def de_batch(emb):
    final_e=[]
    for i in range(len(emb)):
        for j in range(len(emb[i])):
            final_e.append(emb[i][j])
    
    final_e=np.array(final_e, dtype='object')
    return final_e

print("debatching...")

final_train_emb=de_batch(train_emb)
final_test_emb=de_batch(test_emb)
final_blur_emb=de_batch(blur_emb)
#final_deg_emb=de_batch(deg_emb)

debatching...


/tmp/ipykernel_150/2065555189.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  final_e=np.array(final_e, dtype='object')


In [41]:

def final_step(dati):
    temp=[]
    for i in range(len(dati)):
        if i%500==0:
            print(i)
        tmp=[]
        for j in range(len(dati[i])):
            tmp.append(dati[i][j])
        temp.append(np.array(tmp))
    
    temp=np.array(temp)
    
    return temp

print("train...")
fin_train_emb=final_step(final_train_emb)
print("test...")
fin_test_emb=final_step(final_test_emb)
print("blur...")
fin_blur_emb=final_step(final_blur_emb)


train...
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
test...
0
500
1000
1500
2000
2500
3000
blur...
0
500
1000
1500
2000
2500
3000


In [43]:
import h5py

def save_data(path, data):
    with open(path, 'wb') as handle:
        pickle.dump(data, handle)
    print(path)
    print("saved")

def save_embeddings(embedding, true_label,pr_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("E", data=embedding, compression="gzip")    
    fp.create_dataset("Y_predicted", data=pr_label, compression="gzip") 
    fp.create_dataset("Y_original", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")

train_path="Stl_train_emb_v1"
test_path="Stl_test_emb_v1"
blur_path="Stl_new_unseen_emb"
#deg_path="Stl_deg_emb"

save_embeddings(fin_train_emb, train_lab, train_pred, train_path )
save_embeddings(fin_test_emb, test_lab, test_pred, test_path )
save_embeddings(fin_blur_emb, blur_lab, blur_pred, blur_path)
#save_embeddings(fin_deg_emb, deg_lab, deg_pred, deg_path)

#save_data("train_stl.pickle",train_ds)
#save_data("test_stl.pickle",test_ds)
#save_data("blur_stl.pickle",blur_ds)
#save_data("deg_stl.pickle",drift)

print("done")

embeddings saved
embeddings saved
embeddings saved
finito
